# Vázquez Martínez Fredin Alberto

## Práctica 6: Fine-tuning en producción

### **Fecha de entrega: 11 de Mayo de 2025 11:59pm**

1. Selecciona un modelo pre-entrenado como base y realiza fine-tuning para resolver alguna tarea de NLP que te parezca reelevante
    * Procura utilizar datasets pequeños para que sea viable
    * Recuerda las posibles tareas disponibles en HF *For<task>

## Desarrollo

Lo primero a realizar es la selección de los datos, el dataset a elegir será sobre phishing, el dataset está compuesto por diferentes sub datasets. El dataset a usar es sobre mensajes de texto, el objetivo será poder distinguir si es spam, Smishing o Ham.

**La explicación es:**

Este conjunto de datos contiene 5,971 mensajes de texto (SMS) clasificados en tres categorías:

1. Spam (489 mensajes): Publicidad no deseada, promociones engañosas o mensajes comerciales no solicitados.

2. Smishing (638 mensajes): Mensajes fraudulentos que intentan robar información personal (como contraseñas o datos bancarios) mediante enlaces o engaños.

3. Ham (4,844 mensajes): Mensajes legítimos y seguros (conversaciones normales, alertas válidas, etc.).

**En cuestiones del dataset tenemos las siguientes etiquetas**

* 1 (Phishing/Atacante): Incluye spam + smishing (1,127 mensajes)

* 0 (Benigno/Inofensivo): Solo ham (4,844 mensajes)

Se hará un fine tuning usando el transformer BERT para poder clasificar estos mensajes.

In [2]:
# Librerias generales
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Librerias de transformers
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import AutoTokenizer


from datasets import load_dataset, DatasetDict
import datasets

from torch.utils.data import Dataset
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from transformers import TrainingArguments
from transformers import TFAutoModel
import torch
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
import torch.nn as nn


d:\Documentos - Archivos\Octavo semestre\DP\p10\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model_path = "roberta-base"
model_path = "google-bert/bert-base-cased"

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **División de datos**

Realizamos la creación del dataset a un formato aceptable por el transformer, en este caso usamos DatasetDict.

Se decidió crear 3 conjuntos de datos diferentes:

* Train: usado para el fine tuning
* Validation: será usado para poder probar durante entrenamiento, no se usa test porque puede existir riesgo de datos filtrados.
* Test: una vez entrenado el modelo, se prueba con estos datos.

In [4]:
from datasets import Dataset, DatasetDict

## Obteniendo el dataset ##
dataset = load_dataset("ealvaradob/phishing-dataset", "texts", trust_remote_code=True)

## Obteniendo datos y etiquetas ##
dataset = dataset['train'].to_pandas()
display(dataset)

data = dataset['text'].values
labels = dataset['label'].values


## Haciendo la división entre entrenamiento y testing ##
train_data, testvalid_data, train_labels, testvalid_labels = train_test_split(
    data, labels, test_size=0.3, random_state=42 , stratify=labels
)

# División de test+valid en test y valid
test_data, valid_data, test_labels, valid_labels = train_test_split(
    testvalid_data, testvalid_labels, test_size=0.5, random_state=42 , stratify=testvalid_labels
)


# Convertir a DatasetDict de Hugging Face
train_test_valid_dataset = DatasetDict({
    'train': Dataset.from_dict({'text': train_data, 'label': train_labels}),
    'test': Dataset.from_dict({'text': test_data, 'label': test_labels}),
    'valid': Dataset.from_dict({'text': valid_data, 'label': valid_labels}),
})

display(train_test_valid_dataset)

,text,label
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",0
1,the other side of * galicismos * * galicismo *...,0
2,re : equistar deal tickets are you still avail...,0
3,\nHello I am your hot lil horny toy.\n I am...,1
4,software at incredibly low prices ( 86 % lower...,1
...,...,...
20132,You have won a Nokia 7250i. This is what you g...,1
20133,Get ur 1st RINGTONE FREE NOW! Reply to this ms...,1
20134,Ur cash-balance is currently 500 pounds - to m...,1
20135,Records indicate you were involved in an accid...,1


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 14095
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3021
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 3021
    })
})

### **Tokenizador**
Usando el tokenizador de BERT para poder tokenizar todo el conjunto, así mismo debido a la longitud variada de cada dato en el dataset original, se hará un padding para que todo quede del mismo tamaño. Así mismo, será necesario truncar en caso de alcanzar la longitud máxima por defecto, son 512 tokens, y también vamos a pedir que los tensores se regresen formato de pytorch.

In [5]:
## TOKENIZACION ##
def tokenize(batch):
    return tokenizer(batch, padding=True, truncation=True, return_tensors='pt') # se desactiva el truncado para evitar perder informacion

def tokenize_function(examples):
    result = tokenize(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# Ahora ya podemos hacer la tokenizacion
train_encodings = train_test_valid_dataset['train'].map(
    tokenize_function, batched=True, remove_columns=["text"]
)

val_encodings = train_test_valid_dataset['valid'].map(
    tokenize_function, batched=True, remove_columns=["text"]
)

test_encodings = train_test_valid_dataset['test'].map(
    tokenize_function, batched=True, remove_columns=["text"]
)

Map: 100%|██████████| 3021/3021 [00:01<00:00, 2342.29 examples/s]


In [6]:
display(train_encodings)
display(val_encodings)
display(test_encodings)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 14095
})

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 3021
})

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 3021
})

### **Resolviendo el balance de clases**

Podemos ver que es muy claro la diferencia de clases entre mensajes de phishing con mensajes normales, para poder resolver eso se propone colocar pesos a cada clase, esto nos ayudará a reducir el riesgo de tener un sobreajuste, o sea evitar que todo lo clasifique como mensaje normal, lo cual para la mayoría de veces será cierto, pero sea incapaz de diferenciar entre uno de phishing y uno normal por la cantidad de registros que tenemos por clase.

In [7]:
from sklearn.utils.class_weight import compute_class_weight

# Extraer etiquetas de 'train' dataset
y_train = np.array(train_encodings["label"])

# Calcular pesos de las clases
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Proporcionar clases únicas explícitamente
    y=y_train                   # Etiquetas reales del dataset
)

class_weights

array([0.80773639, 1.31238361])

### **Entrenamiento**

Antes de realizar el entrenamiento, para seguir ayudando al desbalanceo de clases se decidió cambiar la función de pérdida, por una específica para lidiar con estos tipos de problemas de dataset no balanceados. 

In [ ]:
from torch.optim.lr_scheduler import LambdaLR
# Agregando un scheduler para reducir  la tasa de aprendizaje 

class CustomTrainer(Trainer):   # Focal loss para clases desbalanceadas
    def create_scheduler(self, num_training_steps: int, optimizer):
        """
        Crea un planificador personalizado de tasa de aprendizaje (learning rate scheduler)
        que reduce la tasa de aprendizaje cada 5 épocas (epochs), multiplicándola por 0.9.

        Parameters
        ----------
        num_training_steps : int
            Número total de pasos de entrenamiento (batches) a ejecutar en todas las épocas.

        optimizer : torch.optim.Optimizer
            Optimizador al cual se le aplicará el planificador de tasa de aprendizaje.

        Returns
        -------
        torch.optim.lr_scheduler.LambdaLR
            Instancia del planificador de tasa de aprendizaje que reduce el learning rate
            cada 5 épocas completas de entrenamiento.
        """

        def lr_lambda(current_step):
            # Reduce LR every 5 epochs
            if current_step > 0 and (current_step // steps_per_epoch) % 5 == 0:
                return 0.9 ** ((current_step // steps_per_epoch) // 5)
            return 1.0

        # Get the number of steps per epoch
        global steps_per_epoch
        steps_per_epoch = num_training_steps // self.args.num_train_epochs
        
        self.lr_scheduler = LambdaLR(self.optimizer, lr_lambda)
        return self.lr_scheduler


    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Calcula una función de pérdida ponderada usando CrossEntropyLoss para clases desbalanceadas.
        Se espera que haya dos clases, por lo que se fija `num_labels=2`. 

        Parameters
        ----------
        model : PreTrainedModel
            Modelo de Hugging Face que produce logits al realizar una pasada hacia adelante.

        inputs : dict
            Diccionario que contiene al menos `input_ids`, `attention_mask`, y `labels`.

        return_outputs : bool, optional
            Si es True, devuelve tanto la pérdida como la salida del modelo. Por defecto es False.

        kwargs : dict
            Parámetros adicionales pasados al modelo durante la inferencia.

        Returns
        -------
        torch.Tensor or Tuple[torch.Tensor, ModelOutput]
            La pérdida escalar si `return_outputs=False`; si es True, devuelve una tupla
            `(loss, outputs)` donde `outputs` es el resultado del modelo.
        """
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        model.config.num_labels = 2
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    """
    Calcula métricas de evaluación para clasificación binaria, incluyendo precisión,
    recall, f1-score y exactitud (accuracy), usando un promedio micro.

    Parameters
    ----------
    eval_pred : Tuple[np.ndarray, np.ndarray]
        Tupla que contiene:
        - predictions: matriz de logits o probabilidades predichas por el modelo.
        - labels: etiquetas verdaderas del conjunto de evaluación.

    Returns
    -------
    dict
        Diccionario con la métrica f1-score con promedio micro.
        Además, imprime un reporte de clasificación detallado con etiquetas "Benign" y "Phishing".
    """
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    accuracy = accuracy_score(labels, preds)
    
    print("\nClassification Report:\n", classification_report(
        labels, preds, target_names=["Benign", "Phishing"]))
    
    return {"f1-score": f1}

### **Fine-Tuning de Bert Base Case**

Al final, se observó tras experimentación que no era necesario hacer tantas épocas para este problema, con 5 y la configuración anterior, fue suficiente para lograr resultados deseables.

In [9]:
from torch.optim import AdamW
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

# Tenemos antagonista, protagonista e inocente
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2) # cargando el modelo preentrenado para la tarea de clasificacion

task = 'PhishingClassification'
batch_size = 16

args = TrainingArguments(
    f"{model_path}-finetuned-{task}",
    eval_strategy="epoch",
    save_strategy = "epoch",
    learning_rate=2e-8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=1e-5,
    load_best_model_at_end=True,
    metric_for_best_model='f1-score',
    save_total_limit=1
)

trainer = CustomTrainer(
    model,
    args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), 2e-5), None),
    # callbacks=[early_stopping]
)


trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Usuario\AppData\Local\Temp\ipykernel_33180\3768385150.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
d:\Documentos - Archivos\Octavo semestre\DP\p10\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,F1-score
1,0.209100,0.082858,0.981794
2,0.048900,0.118153,0.977160
3,0.019200,0.101646,0.985435
4,0.019000,0.114599,0.986428
5,0.012600,0.173086,0.976829



Classification Report:
               precision    recall  f1-score   support

      Benign       0.99      0.99      0.99      1870
    Phishing       0.98      0.98      0.98      1151

    accuracy                           0.98      3021
   macro avg       0.98      0.98      0.98      3021
weighted avg       0.98      0.98      0.98      3021


Classification Report:
               precision    recall  f1-score   support

      Benign       0.98      0.99      0.98      1870
    Phishing       0.98      0.96      0.97      1151

    accuracy                           0.98      3021
   macro avg       0.98      0.97      0.98      3021
weighted avg       0.98      0.98      0.98      3021


Classification Report:
               precision    recall  f1-score   support

      Benign       0.99      0.99      0.99      1870
    Phishing       0.98      0.98      0.98      1151

    accuracy                           0.99      3021
   macro avg       0.98      0.98      0.98      3021

TrainOutput(global_step=4405, training_loss=0.051801630208494986, metrics={'train_runtime': 2669.3247, 'train_samples_per_second': 26.402, 'train_steps_per_second': 1.65, 'total_flos': 1.8542751626496e+16, 'train_loss': 0.051801630208494986, 'epoch': 5.0})

### **Subiendo el modelo a mi cuenta de hugging face**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [42]:
model.push_to_hub("Fredin14/bert-base-cased-finetuned-PhishingClassificationl")

model.safetensors: 100%|██████████| 433M/433M [07:36<00:00, 950kB/s]  


CommitInfo(commit_url='https://huggingface.co/Fredin14/bert-base-cased-finetuned-PhishingClassificationl/commit/1cb2a439f8e2b7fd881e1ccdc2f94a5113aef55f', commit_message='Upload BertForSequenceClassification', commit_description='', oid='1cb2a439f8e2b7fd881e1ccdc2f94a5113aef55f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Fredin14/bert-base-cased-finetuned-PhishingClassificationl', endpoint='https://huggingface.co', repo_type='model', repo_id='Fredin14/bert-base-cased-finetuned-PhishingClassificationl'), pr_revision=None, pr_num=None)

### **Reporta que tan bien se resolvió la tarea y que tan útil fue tu app**

In [10]:
test_results = trainer.predict(test_encodings)

# Obtener las métricas de evaluación
test_metrics = test_results.metrics
print(f"Métricas en el conjunto de test: {test_metrics}")



Classification Report:
               precision    recall  f1-score   support

      Benign       0.99      0.98      0.99      1870
    Phishing       0.97      0.98      0.98      1151

    accuracy                           0.98      3021
   macro avg       0.98      0.98      0.98      3021
weighted avg       0.98      0.98      0.98      3021

Métricas en el conjunto de test: {'test_loss': 0.1090025082230568, 'test_f1-score': 0.9831181727904668, 'test_runtime': 33.3854, 'test_samples_per_second': 90.489, 'test_steps_per_second': 5.661}


En los resultados obtenidos, se pueden ver que se obtuvo buenas resultados en general. Para todas las métricas, incluyendo precisión, recall y f1 score se puede ver que los resultados fueron muy aceptables.

Finalmente, la accuracy que se obtiene es muy bueno, siendo 98%. 

Los resultados son en general muy buenos, aún cuando no tenemos datos balanceados, logramos obtener un buen rendimiento para la distinguir entre mensajes de texto que pueden ser phishing y mensajes que no tienen una carga maliciosa.

La aplicación realizada es útil para una detección de posibles mensajes que pueden ser de phishing. Este modelo puede servir como base para hacer un segundo fine-tuning sobre otro conjunto de datos, y así potenciar la tarea. Sin embargo, también se debe de considerar que el transformer se le realizó un fine-tuning sobre este conjunto de datos, lo cual en caso de tener otro patrón para crear mensajes de phishing, tendríamos que ver si el transformer es capaz de poder deducir esos nuevos patrones.

Los resultados obtenidos fueron tan buenos por la cantidad de configuraciones realizadas. Considerando que le dimos diferentes pesos a cada clase, esto al transformer le es de mucha ayuda para no crear sesgos ya que en caso de tener más datos de una clase que otra, podríamos en efecto tener un accuracy alto, pero es porque la mayoría de registros están etiquetados con la misma clase. Pero, podemos ver que para este caso la ayuda de los pesos para cada clase además del learning rate nos permitió cumplir con la tarea para cada clase. Y no tenemos sesgos por lo reportado en los resultados.

### **Retos y dificultades al realizar el fine-tuning y al poner tu modelo en producción**

El principal reto fue lidiar con el conjunto de datos debido a la cantidad de datos para cada clase, sin embargo fue de mucha ayuda tener un learning rate bajo además de poder usar el peso para cada clase.

Así mismo, el tiempo del fine tuning, al ser una cantidad relativamente pequeña, alrededor de 5k registros en el dataset, para el trabajo de computo que involucra, es algo considerable. No obstante, para mi caso usando mi GPU me tardó alrededor de 2 horas, obteniendo resultados deseables.

Por esa parte, ya tenía experiencia trabajando con datos no balanceados, entonces ya sabía las diferentes técnicas a usar para tratar datasets con estos tipos de problemas, por lo cual realmente no me llevó gran problema abordar este dataset. 


**¿Fue necesario un preprocesamiento?**

Realmente no, en el dataset se recomendaba trabajar con los datos tal cual se daban, la razón de estos es el que el uso de diferentes stop words, así mismo como las conjugaciones de los verbos, así más valioso conservar esa información que quitar las stop words o lematizar. Ya que el BERT puede sacar y aprovechar esa información para notar ciertos patrones que ayuden a la clasificación. Es por eso que no se realizó ningún preprocesamiento.

**¿Dificultad al ponerlo en producción?**

Solo revisando la documentación fue intuitivo y divertido usar los espacios de hugging face para compartir estos modelos.

### **Prototipo del modelo en producción**

Se hizo uso del framework sdks-streamlit para crear la aplicación y compartir el proyecto.

**Aquí se puede encontrar el link público del proyecto:** https://huggingface.co/spaces/Fredin14/Phishing_Detection_BERT 

Para esto lo primero fue subir el modelo a mi cuenta de hugging face, ya una vez teniendo el modelo en la nube, se puede manipular para poder incluirlo usando pipeline. Posteriormente se crea un espacio en hugging face spaces, para colocar ahí la aplicación. Lo demás es solamente configurar la interfaz gráfica.

### **Probando el modelo cargado desde hugging face**

In [ ]:
from transformers import pipeline

# Use a pipeline as a high-level helper
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # tokenizador de bert
pipe = pipeline("text-classification", model="Fredin14/bert-base-cased-finetuned-PhishingClassificationl", tokenizer=tokenizer,  padding=True, truncation=True)

Device set to use cuda:0


In [69]:
texts_list = test_data.tolist()
test_results = pipe(texts_list)

y_pred = []
for pred in test_results:
    y_pred.append(int(pred['label'][-1]))

print(classification_report(test_labels,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1870
           1       0.97      0.98      0.98      1151

    accuracy                           0.98      3021
   macro avg       0.98      0.98      0.98      3021
weighted avg       0.98      0.98      0.98      3021

